# Loading database tables

We can either use a custom `Creator` load from a database to a `DataFrame` or we can use the database engine service itself to output a file which  can then be loaded in `fugue`.

Let's dump some data into a `sqlite` database & read it in `fugue`

In [1]:
import os
import sqlite3

from fugue import FugueWorkflow
import pandas as pd

In [2]:
def create_sqlite_db(path: str, content: str, table: str):
    uri = f"file:{path}"
    lines = content.split("\n")
    headers = lines[0]
    rows = lines[1:]
    with sqlite3.connect(uri, uri=True) as con:
        cur = con.cursor()
        cur.execute(f"CREATE TABLE {table}({headers})")
        values = "(" + '),('.join(row for row in rows) + ")"
        cur.execute(f"INSERT INTO {table} VALUES {values}")
        con.commit()

In [3]:
content = '''\
"TIMESTAMP","RECORD","WS_80m_90deg_Avg","WS_80m_90deg_Std","WS_80m_90deg_3sGust_Max","WS_80m_90deg_Max","WS_80m_270deg_Avg","WS_80m_270deg_Std","WS_80m_270deg_3sGust_Max","WS_80m_270deg_Max","WS_65m_90deg_Avg","WS_65m_90deg_Std","WS_65m_90deg_3sGust_Max","WS_65m_90deg_Max","WS_65m_270deg_Avg","WS_65m_270deg_Std","WS_65m_270deg_3sGust_Max","WS_65m_270deg_Max","WS_50m_90deg_Avg","WS_50m_90deg_Std","WS_50m_90deg_3sGust_Max","WS_50m_90deg_Max","WS_50m_270deg_Avg","WS_50m_270deg_Std","WS_50m_270deg_3sGust_Max","WS_50m_270deg_Max","WS_30m_90deg_Avg","WS_30m_90deg_Std","WS_30m_90deg_3sGust_Max","WS_30m_90deg_Max","WS_30m_270deg_Avg","WS_30m_270deg_Std","WS_30m_270deg_3sGust_Max","WS_30m_270deg_Max","Dir_78m_90deg_avg","Dir_78m_90deg_std","Dir_63m_90deg_avg","Dir_63m_90deg_std","Dir_28m_90deg_avg","Dir_28m_90deg_std","Batt_Volt_Min","Press_Avg","Temp_C80_Avg","Temp_C15_Avg","Hum_Avg"
"2012-05-31 12:20:00",1,1.383,0.6,2.75,3.37,1.368,0.439,2.673,2.74,1.332,0.478,2.75,2.75,1.242,0.379,2.74,2.79,1.162,0.535,2.337,2.75,1.159,0.354,2.34,2.39,1.27,0.614,2.337,2.75,1.322,0.416,2.157,2.24,240.3,46,242,45.39,222,33.45,13.79,1009,13.84,14.08,65.67
"2012-05-31 12:30:00",2,1.183,0.449,1.923,2.13,1.135,0.324,1.94,1.99,0.948,0.524,1.923,2.13,1.068,0.303,1.723,1.74,0.701,0.547,1.923,2.13,0.913,0.308,1.673,1.74,0.771,0.539,1.717,2.13,0.997,0.28,1.657,1.74,282,26.79,264.3,30.25,278.5,62.87,13.73,1009,14.04,14.45,64.51'''

In [4]:
def load_db(table: str, con: str) -> pd.DataFrame:
    return pd.read_sql_table(table, uri)

In [5]:
db = "/tmp/example.db"
uri = f"sqlite:///{db}"
create_sqlite_db(db, content, table="sensors")

with FugueWorkflow() as dag:
    df = dag.create(load_db, params={"table": "sensors", "con": uri})
    df.show()

os.unlink(db)

PandasDataFrame
TIMESTAMP:str |RECORD:long|WS_80m_90deg_Avg:double|WS_80m_90deg_Std:double|WS_80m_90deg_3sGust_Max:double|WS_80m_90deg_Max:double|WS_80m_270deg_Avg:double|WS_80m_270deg_Std:double|WS_80m_270deg_3sGust_Max:double|WS_80m_270deg_Max:double|WS_65m_90deg_Avg:double|WS_65m_90deg_Std:double|WS_65m_90deg_3sGust_Max:double|WS_65m_90deg_Max:double|WS_65m_270deg_Avg:double|WS_65m_270deg_Std:double|WS_65m_270deg_3sGust_Max:double|WS_65m_270deg_Max:double|WS_50m_90deg_Avg:double|WS_50m_90deg_Std:double|WS_50m_90deg_3sGust_Max:double|WS_50m_90deg_Max:double|WS_50m_270deg_Avg:double|WS_50m_270deg_Std:double|WS_50m_270deg_3sGust_Max:double|WS_50m_270deg_Max:double|WS_30m_90deg_Avg:double|WS_30m_90deg_Std:double|WS_30m_90deg_3sGust_Max:double|WS_30m_90deg_Max:double|WS_30m_270deg_Avg:double|WS_30m_270deg_Std:double|WS_30m_270deg_3sGust_Max:double|WS_30m_270deg_Max:double|Dir_78m_90deg_avg:double|Dir_78m_90deg_std:double|Dir_63m_90deg_avg:double|Dir_63m_90deg_std:double|Dir_28m_90deg_avg